In [1]:
### imports

# external modules
import os
from os.path import exists
import os.path
import pandas as pd
import json
import sys
import itertools
import numpy as np
import matplotlib.pyplot as plt
import importlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import HistStruct
importlib.reload(HistStruct)
import SubHistStruct
importlib.reload(SubHistStruct)
import DataLoader
importlib.reload(DataLoader)
import AutoEncoder
importlib.reload(AutoEncoder)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)

2022-06-30 16:36:51.859713: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.1-8d929/x86_64-centos7-gcc8-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.1-f3599/x86_64-centos7-gcc8-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_forest:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/python/framework:/cvmfs/sft.cern.ch/lcg/releases/java/8u222-884d8/x86_64-centos7-gcc8-opt/jre/lib/amd64:/cvmfs/sft.

<module 'HyperRectangleFitter' from '/eos/home-i01/k/khowey/SWAN_projects/ML4DQMDC-PixelAE/KH-AutoencoderTest/../src/cloudfitters/HyperRectangleFitter.py'>

The next 5 cells are only necessary if a JSON does not already exist. If one does, skip to the read in JSON cell

In [2]:
### Histname identification
# Here is where all the available histnames should be entered, in the form of a histslist generated by the permutations notebook

histnames = [['chargeInner_PXLayer_1', 'chargeInner_PXLayer_2', 'chargeInner_PXLayer_3', 'chargeInner_PXLayer_4', 'chargeOuter_PXLayer_1', 'chargeOuter_PXLayer_2', 'chargeOuter_PXLayer_3', 'chargeOuter_PXLayer_4', 'adc_PXLayer_1', 'adc_PXLayer_2', 'adc_PXLayer_3', 'adc_PXLayer_4', 'size_PXLayer_1', 'size_PXLayer_2', 'size_PXLayer_3', 'size_PXLayer_4'], ['charge_PXDisk_+1', 'charge_PXDisk_+2', 'charge_PXDisk_+3', 'adc_PXDisk_+1', 'adc_PXDisk_+2', 'adc_PXDisk_+3', 'size_PXDisk_+1', 'size_PXDisk_+2', 'size_PXDisk_+3'], ['charge_PXDisk_-1', 'charge_PXDisk_-2', 'charge_PXDisk_-3', 'adc_PXDisk_-1', 'adc_PXDisk_-2', 'adc_PXDisk_-3', 'size_PXDisk_-1', 'size_PXDisk_-2', 'size_PXDisk_-3'], ['NormalizedHitResiduals_TIB__Layer__1', 'NormalizedHitResiduals_TIB__Layer__2', 'NormalizedHitResiduals_TIB__Layer__3', 'NormalizedHitResiduals_TIB__Layer__4', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__1', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__2', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__3', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__4', 'Summary_TotalNumberOfDigis__TIB__layer__1', 'Summary_TotalNumberOfDigis__TIB__layer__2', 'Summary_TotalNumberOfDigis__TIB__layer__3', 'Summary_TotalNumberOfDigis__TIB__layer__4'], ['NormalizedHitResiduals_TOB__Layer__1', 'NormalizedHitResiduals_TOB__Layer__2', 'NormalizedHitResiduals_TOB__Layer__3', 'NormalizedHitResiduals_TOB__Layer__4', 'NormalizedHitResiduals_TOB__Layer__5', 'NormalizedHitResiduals_TOB__Layer__6', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__1', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__2', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__3', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__4', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__5', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__6', 'Summary_TotalNumberOfDigis__TOB__layer__1', 'Summary_TotalNumberOfDigis__TOB__layer__2', 'Summary_TotalNumberOfDigis__TOB__layer__3', 'Summary_TotalNumberOfDigis__TOB__layer__4', 'Summary_TotalNumberOfDigis__TOB__layer__5', 'Summary_TotalNumberOfDigis__TOB__layer__6'], ['Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__3', 'Summary_TotalNumberOfDigis__TID__PLUS__wheel__1', 'Summary_TotalNumberOfDigis__TID__PLUS__wheel__2', 'Summary_TotalNumberOfDigis__TID__PLUS__wheel__3'], ['NormalizedHitResiduals_TID__wheel__1', 'NormalizedHitResiduals_TID__wheel__2', 'NormalizedHitResiduals_TID__wheel__3', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__3', 'Summary_TotalNumberOfDigis__TID__MINUS__wheel__1', 'Summary_TotalNumberOfDigis__TID__MINUS__wheel__2', 'Summary_TotalNumberOfDigis__TID__MINUS__wheel__3'], ['Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__3', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__4', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__5', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__6', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__7', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__8', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__9', 'Summary_TotalNumberOfDigis__TEC__PLUS__wheel__1', 'Summary_TotalNumberOfDigis__TEC__PLUS__wheel__2', 'Summary_TotalNumberOfDigis__TEC__PLUS__wheel__3', 'Summary_TotalNumberOfDigis__TEC__PLUS__wheel__4', 'Summary_TotalNumberOfDigis__TEC__PLUS__wheel__5', 'Summary_TotalNumberOfDigis__TEC__PLUS__wheel__6', 'Summary_TotalNumberOfDigis__TEC__PLUS__wheel__7', 'Summary_TotalNumberOfDigis__TEC__PLUS__wheel__8', 'Summary_TotalNumberOfDigis__TEC__PLUS__wheel__9'], ['NormalizedHitResiduals_TEC__wheel__1', 'NormalizedHitResiduals_TEC__wheel__2', 'NormalizedHitResiduals_TEC__wheel__3', 'NormalizedHitResiduals_TEC__wheel__4', 'NormalizedHitResiduals_TEC__wheel__5', 'NormalizedHitResiduals_TEC__wheel__6', 'NormalizedHitResiduals_TEC__wheel__7', 'NormalizedHitResiduals_TEC__wheel__8', 'NormalizedHitResiduals_TEC__wheel__9', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__3', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__4', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__5', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__6', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__7', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__8', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__9', 'Summary_TotalNumberOfDigis__TEC__MINUS__wheel__1', 'Summary_TotalNumberOfDigis__TEC__MINUS__wheel__2', 'Summary_TotalNumberOfDigis__TEC__MINUS__wheel__3', 'Summary_TotalNumberOfDigis__TEC__MINUS__wheel__4', 'Summary_TotalNumberOfDigis__TEC__MINUS__wheel__5', 'Summary_TotalNumberOfDigis__TEC__MINUS__wheel__6', 'Summary_TotalNumberOfDigis__TEC__MINUS__wheel__7', 'Summary_TotalNumberOfDigis__TEC__MINUS__wheel__8', 'Summary_TotalNumberOfDigis__TEC__MINUS__wheel__9'], ['NumberOfTracks_lumiFlag_GenTk', 'NumberOfRecHitsPerTrack_lumiFlag_GenTk', 'Chi2oNDF_lumiFlag_GenTk', 'goodvtxNbr'], ['NumberOfClustersInPixel', 'num_clusters_ontrack_PXBarrel', 'num_clusters_ontrack_PXForward', 'NumberOfClustersInStrip']]

In [3]:
### Data Controls and Selection - 1D Autoncoder

# The directory data is located in
datadir = '../data/'

years= ['2017']
eras = ['B', 'C' , 'D', 'F']

# Select a list of good runs to train on in development training_mode
# Should be validated by eye
trainrunsls = {'2017B':{
                    "297057":[[-1]], 
                    "297099":[[-1]], 
                    "297101":[[-1]],
                    "297113":[[-1]], 
                    "297114":[[-1]], 
                    "297175":[[-1]],   # A decently clean histogram
                    "297177":[[-1]],
                    "297179":[[-1]], 
                    "297215":[[-1]],
                    "297218":[[-1]],
                    "297225":[[-1]],
                    "297296":[[-1]], 
                    "297411":[[-1]],
                    "297426":[[-1]],  
                    "297431":[[-1]],
                    "297434":[[-1]], 
                    "297468":[[-1]],
                    "297483":[[-1]],
                    "297486":[[-1]],
                    "297503":[[-1]],
                    "297557":[[-1]],
                    "297598":[[-1]],
                    "297604":[[-1]],   # A decently clean histogram
                    "297620":[[-1]],   # A decently clean histogram
                    "297659":[[-1]],   # An okay histogram
                    "297670":[[-1]],   # A decently clean histogram
                    "297674":[[-1]],
                    "297678":[[-1]],   # A particularly messy histogram
                    "297722":[[-1]],   # A decently clean histogram
                    "298997":[[-1]],
                    "299061":[[-1]],
                    "299065":[[-1]],   # A decently clean histogram
                    "299067":[[-1]],   # A decently clean histogram
                    "299096":[[-1]],
                    "299149":[[-1]],
                    "299178":[[-1]],   # A decently clean histogram
                    "299184":[[-1]],   # A particularly messy histogram
                    "299185":[[-1]],   # A decently clean histogram
                    "299327":[[-1]],
                    "299329":[[-1]], 
                    "299480":[[-1]]    # A decently clean histogram
                    },
                   '2017C': {
                      "299370":[[-1]],
                      "299394":[[-1]],
                      "299420":[[-1]],
                      "299477":[[-1]],
                      "299593":[[-1]],
                      "299597":[[-1]],
                      "299617":[[-1]],
                      "300018":[[-1]],
                      "300105":[[-1]],
                      "300117":[[-1]],
                      "300124":[[-1]],
                      "300234":[[-1]],
                      "300237":[[-1]],
                      "300240":[[-1]],
                      "300370":[[-1]],
                      "300157":[[-1]],
                      "300373":[[-1]],
                      "300392":[[-1]],
                      "300395":[[-1]],
                      "300401":[[-1]],
                      "300462":[[-1]],
                      "300466":[[-1]],
                      "300514":[[-1]],
                      "300517":[[-1]],
                      "300538":[[-1]],
                      "300539":[[-1]],
                      "300364":[[-1]],
                 },'2017F':{
                      "305310":[[-1]],
                      "305040":[[-1]],
                      "305043":[[-1]],
                      "305185":[[-1]],
                      "305204":[[-1]],
                      "305234":[[-1]],
                      "305247":[[-1]],
                      "305313":[[-1]],
                      "305338":[[-1]],
                      "305350":[[-1]],
                      "305364":[[-1]],
                      "305376":[[-1]],
                      "306042":[[-1]],
                      "306051":[[-1]],
                      "305406":[[-1]],
                      "306122":[[-1]],
                      "306134":[[-1]],
                      "306137":[[-1]],
                      "306154":[[-1]],
                      "306170":[[-1]],
                      "306417":[[-1]],
                      "306432":[[-1]],
                      "306456":[[-1]],
                      "305516":[[-1]],
                      "305586":[[-1]],
                      "305588":[[-1]],
                      "305590":[[-1]],
                      "305809":[[-1]],
                      "305832":[[-1]],
                      "305840":[[-1]],
                      "305898":[[-1]],
                      "306029":[[-1]],
                      "306037":[[-1]],
                      "306095":[[-1]],
                   },
                '2018':{ # needs to be re-checked, not guaranteed to be fully correct or representative.   
                  "315267":[[-1]] 
}}

badrunsls = {'2017B':
                {
                    "297048":[[-1]],
                    "297282":[[-1]],
                    "297283":[[-1]],
                    "297284":[[-1]],
                    "297287":[[-1]],
                    "297288":[[-1]],
                    "297289":[[-1]],
                    "299316":[[-1]],
                    "299317":[[-1]],
                    "299318":[[-1]],
                    "299324":[[-1]],
                    "299326":[[-1]],
                    "301086":[[88,126]],
                    "301086":[[89,89]],
                    "303948":[[1710,1710]],
                    "297047":[[-1]], #close but, true bad for all 8
                    "297169":[[-1]], #true bad for all 8
                    "297211":[[-1]], #Reconstructs well
                    "299325":[[-1]], #Reconstructs well
                    "297664":[[-1]], #true bad for all 8
                    "299317":[[-1]], #true bad for all 8
                    "297169":[[-1]], #true bad for all 8
                    "297502":[[-1]]
                },
             '2017C':{
                 "300781":[[-1]], # bad for tracking (pixels were excluded.
                 "300079":[[-1]], # is bad for strips and then also for tracking
                 "302029":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good 
                 "300576":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
                 "300574":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
                 "300282":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
                 "301912":[[-1]], # Half bad for pixels (lost HV or readout card)  
                 "301086":[[-1]], # Half bad for pixels (lost HV or readout card)  
                 "300283":[[-1]], # Half bad for pixels (lost HV or readout card) 
                 "300282":[[-1]], # Half bad for pixels (lost HV or readout card) 
                 "300281":[[-1]], # Half bad for pixels (lost HV or readout card) 
                 "300239":[[-1]], # Half bad for pixels (lost HV or readout card)
                 "301394":[[-1]], # Marginal for pixels
                 "301183":[[-1]], # Marginal for pixels
                 "300398":[[-1]], # Marginal for pixels
                 "300389":[[-1]], # Marginal for pixels
                 "300365":[[-1]]  # Marginal for pixels
              },
             '2017E':{
                 "304740":[[-1]], # Bad for pixels and tracking - holes in PXLayer 1
                 "304776":[[-1]], # Bad for pixels and tracking - holes in PXLayer 1
                 "304506":[[-1]], # Portcard problem for pixels
                 "304507":[[-1]], # Portcard problem for pixels 
                 "303989":[[-1]], # Bad for pixels, power supply died
                 "303824":[[-1]]  # Partly bad for strips due to a test
             },
             '2017F':{
                 "306422":[[-1]], # Partly bad for strips - 2 data readouts failed 
                 "306423":[[-1]], # Partly bad for strips - 2 data readouts failed
                 "306425":[[-1]], # Partly bad for strips - 2 data readouts failed
                 "305440":[[-1]], # Partly bad for strips - 1 data readout failed
                 "305441":[[-1]], # Partly bad for strips - 1 data readout failed
                 "305249":[[-1]], # Bad for pixels - half of disk failed 
                 "305250":[[-1]], # Bad for pixels - half of disk failed
                 "305064":[[-1]], # Marginal for pixels - some readout failed
             },
             '2018': # needs to be re-checked, not guaranteed to be fully correct or representative.
                {
                "317479":[[-1]],
                "317480":[[-1]],
                "317481":[[-1]],
                "317482":[[-1]],
                "319847":[[1,35]]
            }}

In [ ]:
### Read in runs data from CSV files
runsList = []
dloader = DataLoader.DataLoader()
for year in years:
    for era in eras:
        print('\nNow processing for ' + year + era)
        for histgroup in histnames:
            for histname in histgroup:
                filename = datadir + year + era + '/DF' + year + era + '_' + histname + '.csv'
                print('Adding ' + filename)
                df = dloader.get_dataframe_from_file( filename )
                df = dfu.rm_duplicates(df)
                runsList.append(dfu.get_runs(df))


Now processing for 2017B
Adding ../data/2017B/DF2017B_chargeInner_PXLayer_1.csv
Adding ../data/2017B/DF2017B_chargeInner_PXLayer_2.csv
Adding ../data/2017B/DF2017B_chargeInner_PXLayer_3.csv
Adding ../data/2017B/DF2017B_chargeInner_PXLayer_4.csv
Adding ../data/2017B/DF2017B_chargeOuter_PXLayer_1.csv
Adding ../data/2017B/DF2017B_chargeOuter_PXLayer_2.csv
Adding ../data/2017B/DF2017B_chargeOuter_PXLayer_3.csv
Adding ../data/2017B/DF2017B_chargeOuter_PXLayer_4.csv
Adding ../data/2017B/DF2017B_adc_PXLayer_1.csv
Adding ../data/2017B/DF2017B_adc_PXLayer_2.csv
Adding ../data/2017B/DF2017B_adc_PXLayer_3.csv
Adding ../data/2017B/DF2017B_adc_PXLayer_4.csv
Adding ../data/2017B/DF2017B_size_PXLayer_1.csv
Adding ../data/2017B/DF2017B_size_PXLayer_2.csv
Adding ../data/2017B/DF2017B_size_PXLayer_3.csv
Adding ../data/2017B/DF2017B_size_PXLayer_4.csv
Adding ../data/2017B/DF2017B_charge_PXDisk_+1.csv
Adding ../data/2017B/DF2017B_charge_PXDisk_+2.csv
Adding ../data/2017B/DF2017B_charge_PXDisk_+3.csv
Addi

Adding ../data/2017B/DF2017B_Summary_TotalNumberOfDigis__TEC__MINUS__wheel__2.csv
Adding ../data/2017B/DF2017B_Summary_TotalNumberOfDigis__TEC__MINUS__wheel__3.csv
Adding ../data/2017B/DF2017B_Summary_TotalNumberOfDigis__TEC__MINUS__wheel__4.csv
Adding ../data/2017B/DF2017B_Summary_TotalNumberOfDigis__TEC__MINUS__wheel__5.csv
Adding ../data/2017B/DF2017B_Summary_TotalNumberOfDigis__TEC__MINUS__wheel__6.csv
Adding ../data/2017B/DF2017B_Summary_TotalNumberOfDigis__TEC__MINUS__wheel__7.csv
Adding ../data/2017B/DF2017B_Summary_TotalNumberOfDigis__TEC__MINUS__wheel__8.csv
Adding ../data/2017B/DF2017B_Summary_TotalNumberOfDigis__TEC__MINUS__wheel__9.csv
Adding ../data/2017B/DF2017B_NumberOfTracks_lumiFlag_GenTk.csv
Adding ../data/2017B/DF2017B_NumberOfRecHitsPerTrack_lumiFlag_GenTk.csv
Adding ../data/2017B/DF2017B_Chi2oNDF_lumiFlag_GenTk.csv
Adding ../data/2017B/DF2017B_goodvtxNbr.csv
Adding ../data/2017B/DF2017B_NumberOfClustersInPixel.csv
Adding ../data/2017B/DF2017B_num_clusters_ontrack_P

Adding ../data/2017C/DF2017C_NormalizedHitResiduals_TEC__wheel__6.csv
Adding ../data/2017C/DF2017C_NormalizedHitResiduals_TEC__wheel__7.csv
Adding ../data/2017C/DF2017C_NormalizedHitResiduals_TEC__wheel__8.csv
Adding ../data/2017C/DF2017C_NormalizedHitResiduals_TEC__wheel__9.csv
Adding ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__1.csv
Adding ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__2.csv
Adding ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__3.csv
Adding ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__4.csv
Adding ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__5.csv
Adding ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__6.csv
Adding ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__7.csv
Adding ../data/2017C/DF2017C_Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__8.csv
Adding ../data/2

Adding ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TEC__PLUS__wheel__1.csv
Adding ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TEC__PLUS__wheel__2.csv
Adding ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TEC__PLUS__wheel__3.csv
Adding ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TEC__PLUS__wheel__4.csv
Adding ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TEC__PLUS__wheel__5.csv
Adding ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TEC__PLUS__wheel__6.csv
Adding ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TEC__PLUS__wheel__7.csv
Adding ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TEC__PLUS__wheel__8.csv
Adding ../data/2017D/DF2017D_Summary_TotalNumberOfDigis__TEC__PLUS__wheel__9.csv
Adding ../data/2017D/DF2017D_NormalizedHitResiduals_TEC__wheel__1.csv
Adding ../data/2017D/DF2017D_NormalizedHitResiduals_TEC__wheel__2.csv
Adding ../data/2017D/DF2017D_NormalizedHitResiduals_TEC__wheel__3.csv
Adding ../data/2017D/DF2017D_NormalizedHitResiduals_TEC__whee

Adding ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__3.csv
Adding ../data/2017F/DF2017F_Summary_TotalNumberOfDigis__TID__MINUS__wheel__1.csv
Adding ../data/2017F/DF2017F_Summary_TotalNumberOfDigis__TID__MINUS__wheel__2.csv
Adding ../data/2017F/DF2017F_Summary_TotalNumberOfDigis__TID__MINUS__wheel__3.csv
Adding ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__1.csv
Adding ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__2.csv
Adding ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__3.csv
Adding ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__4.csv
Adding ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__5.csv
Adding ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__6.csv
Adding ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__7.csv
Adding ../data/2017F/DF2017F_Summary_ClusterStoNCorr__OnT

In [ ]:
### Compare CSV run data to find which runs are present everywhere

completeRuns = []
incomplete = []
for i,runs in enumerate(runsList):
    for run in runs:
        complete = True
        for j in range(i, len(runsList)):
            if run not in runsList[j]:
                complete = False
                break
        if complete: 
            completeRuns.append(run)
        else:
            incomplete.append(run)

print(incomplete)

In [21]:
### Save to a JSON file so the process can be faster next time

runsls = {}
for run in completeRuns:
    runls = {}
    runls[str(run)] = [[-1]]
    runsls.update(runls)

print(runsls)
    
jsonu.writejson(runsls, '../data/Complete_Runs', overwrite=True)

{'305040': [[-1]], '305043': [[-1]], '305044': [[-1]], '305045': [[-1]], '305046': [[-1]], '305059': [[-1]], '305062': [[-1]], '305063': [[-1]], '305064': [[-1]], '305081': [[-1]], '305112': [[-1]], '305113': [[-1]], '305114': [[-1]], '305178': [[-1]], '305179': [[-1]], '305180': [[-1]], '305181': [[-1]], '305182': [[-1]], '305183': [[-1]], '305184': [[-1]], '305185': [[-1]], '305186': [[-1]], '305188': [[-1]], '305202': [[-1]], '305204': [[-1]], '305207': [[-1]], '305208': [[-1]], '305234': [[-1]], '305236': [[-1]], '305237': [[-1]], '305247': [[-1]], '305248': [[-1]], '305249': [[-1]], '305250': [[-1]], '305252': [[-1]], '305282': [[-1]], '305310': [[-1]], '305311': [[-1]], '305312': [[-1]], '305313': [[-1]], '305314': [[-1]], '305336': [[-1]], '305338': [[-1]], '305341': [[-1]], '305349': [[-1]], '305350': [[-1]], '305351': [[-1]], '305358': [[-1]], '305364': [[-1]], '305365': [[-1]], '305366': [[-1]], '305376': [[-1]], '305377': [[-1]], '305405': [[-1]], '305406': [[-1]], '305440':

At this point, we can just load in runsls

In [ ]:
### Reload presaved data - ONLY WORKS IF JSON WAS PREVIOUSLY CREATED

runsls = jsonu.loadjson( '../data/Complete_Runs.json' )

In [7]:
### Select out the runs we're interested in - NOTE: Requires tranrunsls to be initialized in cell 3

print(runsls['305040'])

usable = []
for key in trainrunsls['2017C']:
    try:
        runsls[key]
        usable.append(key)
    except:
        continue

[[-1]]
[]
